In [1]:
import pandas, numpy, os, seaborn, librosa

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
import torchaudio


### Dataset-> Toronto Emotional Speech dataset

In [ ]:
paths=[]
labels=[]